**ETF TAX Calculator AUSTRIA**

Data provided by https://my.oekb.at/kapitalmarkt-services/kms-output/fonds-info/sd/af/f?isin=IE00BFY0GT14

Calculation is made in the following way :
$$
\begin{align*}
\text{Capital\_Gain} &= \text{Österreichische\_KESt} \times \text{\#Shares} \\
\text{Taxable\_ETF\_Gain} &= \text{Value\_Year\_After} - \text{Value\_Year\_Before}  \\
\text{Tax\_Paid} &=  \frac {\text{Capital\_Gain}}{\text{USDEUR}} \\
\text{Percentage\_Tax\_Paid} &= \frac {\text{Österreichische KESt} \times 100}{\text{Taxable\_ETF\_Gain} \times \text{USDEUR}}
\end{align*}
$$

Where:


* {Österreichische KESt} is the value provided by the Fund to Öekb (USD).
* Value_Year_After  (USD) is the ETF value at the date when KESt is provided.
* Value_Year_Before (USD) is the ETF value one year before.
* USDEUR is computed at the date when KESt is provided.

In [1]:
from modules import general_utils
from modules import stock_utils
import pandas as pd
from pandas import DataFrame
import warnings

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains # Importa ActionChains
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import os

def insert_isin_and_get_results(url, chrome_path, chromedriver_path, isin):
    """Inserisce ISIN, gestisce i cookie, clicca il pulsante e restituisce i risultati della tabella."""

    try:
        
        service = Service(executable_path=chromedriver_path)

        options = Options()
        options.binary_location = chrome_path
        # options.add_argument('--headless=new')
        
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        # Gestione del banner dei cookie (con XPath corretto e scroll)
        try:
            # Attesa esplicita per il banner
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//*[@id='BorlabsCookieBox']/div/div/div/div[1]")))

            cookie_button = WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.XPATH, "//*[@id='CookieBoxSaveButton']")) # XPath corretto
            )
            actions = ActionChains(driver)
            actions.move_to_element(cookie_button).perform()
            cookie_button.click()

            WebDriverWait(driver, 30).until(EC.invisibility_of_element_located((By.ID, "CookieBoxContainer"))) #attesa che sparisca il div
        except (TimeoutException, NoSuchElementException) as e:
            print(f"Errore gestione Cookie: {e}")
            driver.save_screenshot("cookie_error.png")

        # Inserisci l'ISIN
        isin_input = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, "isin"))
        )
        isin_input.clear()
        isin_input.send_keys(isin)
        
        # Clicca il pulsante "Nettorendite berechnen"
        button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, "//*[@id='button']"))
        )
        button.click()

       # Attesa esplicita più lunga per la tabella
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='tablepress-etf_rendite_rechner_tabelle_details_wrapper']"))
            )

        # Salta la prima riga durante l'estrazione
        # Estrai i dati dalla tabella
        table = driver.find_element(By.XPATH, "//*[@id='tablepress-etf_rendite_rechner_tabelle_details_wrapper']")
        tbody = driver.find_element(By.TAG_NAME, "tbody")  # Trova il tbody

        data = []
        for row in tbody.find_elements(By.TAG_NAME, "tr"):  # Itera sulle righe del tbody
            row_data = []
            for cell in row.find_elements(By.TAG_NAME, "td"):  # Itera sulle celle di ogni riga
                cell_text = cell.text.strip()
                row_data.append(cell_text if cell_text else "-")  # Gestione celle vuote
            data.append(row_data)

        # Stampa i dati
        for row in data:
            print(row)


        # Stampa i dati della tabella
        for row in data:
            print(row)

    except TimeoutException as e:

        
        print(f"Timeout: {e}")
        #if 'driver' in locals():
            #driver.save_screenshot("timeout.png")
    except Exception as e:
        print(f"Errore generico: {e}")
        #if 'driver' in locals():
            #driver.save_screenshot("errore_generico.png")
    finally:
        if 'driver' in locals():
            driver.quit()


chrome_path = r"C:\Users\david\Downloads\chrome-win64\chrome-win64\chrome.exe"
chromedriver_path = r"C:\Users\david\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"
isin_to_insert = "IE00BFY0GT14"
url = "https://www.pfennigfuchser.at/der-etf-rendite-rechner/"

insert_isin_and_get_results(url, chrome_path, chromedriver_path, isin_to_insert)


['ETF-ISIN', '-']
['ETF-ISIN', '-']
